In [1]:
# This changes the current directory to the base saga directory - make sure to run this first!
# This is necessary to be able to import the py files and use the right directories,
# while keeping all the notebooks in their own directory.
import os
import sys

if 'saga_base_dir' not in locals():
    saga_base_dir = os.path.abspath('..')
if saga_base_dir not in sys.path:
    os.chdir(saga_base_dir)

In [21]:
from __future__ import division, print_function

from astropy import units as u

In [2]:
%matplotlib inline

from matplotlib import pyplot as plt

# Hosts 

Hosts are identified in the [HSC overlap check](HSC%20overlap%20check.ipynb) notebook.  For DR1 they are (in "300 kpc" circles):

* Alice: 75%
* Othello: 66%
* Dune: 100%

In [3]:
import hosts

In [7]:
hostobjs = hosts.get_saga_hosts_from_google()
hschosts = tuple([h for h in hostobjs if h.name in ('Alice', 'Othello', 'Dune')])
assert len(hschosts) == 3
hschosts

Using cached version of google hosts list from file "hosts_dl.pkl2"


(<hosts.NSAHost object w/ name 'Alice' AKA: ['NGC4030', 'NSA140594']>,
 <hosts.NSAHost object w/ name 'Dune' AKA: ['NGC5750', 'NSA165536']>,
 <hosts.NSAHost object w/ name 'Othello' AKA: ['NGC5792', 'NSA145729']>)

In [36]:
for h in hschosts:
    h.hscfn = os.path.join('catalogs', 'hsc_pdr1_{}.csv.gz'.format(h.name))

# Generate queries 

These queries are meant for the HSC casjobs at https://hsc-release.mtk.nao.ac.jp/datasearch/ (which requires login)

In [33]:
hsc_qry_templ="""
SELECT {cols}
FROM {table}
WHERE coneSearch(coord, {hra}, {hdec}, {radius})
"""[1:-1]

bands = 'gri'
cols = 'object_id,ra,dec'.split(',')
for band in bands:
    magcol_to_add = []
    magcol_to_add.append('{}cmodel_mag'.format(band))
    magcol_to_add.append('{}mag_psf'.format(band))
    magcol_to_add.append('{}mag_kron'.format(band))
    magcol_to_add.append('{}mag_aperture10'.format(band))
    magcol_to_add.append('{}mag_aperture20'.format(band))
    magcol_to_add.append('{}mag_aperture30'.format(band))
    for magcol in magcol_to_add:
        cols.append(magcol)
        cols.append(magcol+'_err')
    
    cols.append('{}flux_kron_radius'.format(band))
    cols.append('a_{}'.format(band))
cols = ','.join(cols)

In [37]:
table_to_query='pdr1_wide.forced'

for h in hschosts:
    print('Query for host', h.name, 'which should be saved to', h.hscfn)
    qry = hsc_qry_templ.format(table=table_to_query, cols=cols,
                               hra=h.ra, hdec=h.dec, 
                               radius=h.environsarcmin*u.arcmin.to(u.arcsec))
    print(qry, '\n')

Query for host Alice which should be saved to catalogs/hsc_pdr1_Alice.csv.gz
SELECT object_id,ra,dec,gcmodel_mag,gcmodel_mag_err,gmag_psf,gmag_psf_err,gmag_kron,gmag_kron_err,gmag_aperture10,gmag_aperture10_err,gmag_aperture20,gmag_aperture20_err,gmag_aperture30,gmag_aperture30_err,gflux_kron_radius,a_g,rcmodel_mag,rcmodel_mag_err,rmag_psf,rmag_psf_err,rmag_kron,rmag_kron_err,rmag_aperture10,rmag_aperture10_err,rmag_aperture20,rmag_aperture20_err,rmag_aperture30,rmag_aperture30_err,rflux_kron_radius,a_r,icmodel_mag,icmodel_mag_err,imag_psf,imag_psf_err,imag_kron,imag_kron_err,imag_aperture10,imag_aperture10_err,imag_aperture20,imag_aperture20_err,imag_aperture30,imag_aperture30_err,iflux_kron_radius,a_i
FROM pdr1_wide.forced
WHERE coneSearch(coord, 180.09842901, -1.10007802185, 2691.60292683) 

Query for host Dune which should be saved to catalogs/hsc_pdr1_Dune.csv.gz
SELECT object_id,ra,dec,gcmodel_mag,gcmodel_mag_err,gmag_psf,gmag_psf_err,gmag_kron,gmag_kron_err,gmag_aperture10,gmag_